In [1]:
import numpy as np
import pandas as pd
import os
import sys
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
from PIL import Image
from keras import utils
%matplotlib inline

Using TensorFlow backend.


In [2]:
train_df = pd.read_csv('../data/train.csv')

In [3]:
#get targets to write out
y_train = train_df['sirna'].values
y_train = utils.to_categorical(y_train)
print(y_train.shape)

(36515, 1108)


In [5]:
def parse_all_train_2():
    dataset = 'train'
    experiments = train_df['experiment'].values
    plates = train_df['plate'].values
    wells = train_df['well'].values
    sites = np.array([1,2])
    channels=np.array([1,2,3,4,5,6])
    
    x_id = []
    y_id = []
    
    for site in sites:
        for i in tqdm_notebook(range(36515)):
            site_img = np.empty((512, 512, 6), dtype=np.float16)
            for channel in channels:
                path = f'../data/{dataset}/{experiments[i]}/Plate{plates[i]}/{wells[i]}_s{site}_w{channel}.png'
                site_img[:, :, channel-1] = Image.open(path)
            
            #normalize
            site_img /= 255
            
            #write out the image
            np.save(f'../data/train_parsed/x_{experiments[i]}_{plates[i]}_{wells[i]}_s{site}', site_img, allow_pickle=True)
            np.save(f'../data/train_parsed/y_{experiments[i]}_{plates[i]}_{wells[i]}_s{site}', y_train[i], allow_pickle=True)
            
            x_id.append(f'x_{experiments[i]}_{plates[i]}_{wells[i]}_s{site}')
            y_id.append(f'y_{experiments[i]}_{plates[i]}_{wells[i]}_s{site}')
    
    return x_id, y_id

In [6]:
x_id, y_id = parse_all_train_2()

np.save('../data/x_id', x_id, allow_pickle=True)
np.save('../data/y_id', y_id, allow_pickle=True)